# Gathering player Wikipedia data

## Part 1: Player Wikipedia pages

In [16]:
import csv
import requests
import time

playerinfo = []
with open("USWNT/playerdata.csv", "r", newline = "") as infile:
    csvin = csv.reader(infile)
    headers = next(csvin)
    for line in csvin:
        playerinfo.append(line)

filename = headers.index("Filename")
firstname = headers.index("Firstname")
surname = headers.index("Surname")

wikidata = []

for singleplayer in playerinfo:
    wikidatarow = []
    playerfilename = singleplayer[filename]
    playerfirstname = singleplayer[firstname]
    playersurname = singleplayer[surname]

    # Make Players' Wikipedia URLs
    if playersurname == "Campbell" or playersurname == "Smith" or playersurname == "Williams" or playersurname == "Fox":
        wikiplayerurl = "https://en.wikipedia.org/wiki/" + playerfirstname + "_" + playersurname + "_(soccer)"
    else:
        wikiplayerurl = "https://en.wikipedia.org/wiki/" + playerfirstname + "_" + playersurname
    
    # Make filenames to write the HTML from the Wikipedia pages
    last_slash = wikiplayerurl.rindex("/")
    playerwikifilename = "PlayerWiki" + wikiplayerurl[last_slash:] + ".html"
    
    # Request HTML from Wikipedia
    outfile = open(playerwikifilename, "w")
    result = requests.get(wikiplayerurl)
    print(result.text, file=outfile)
    outfile.close()
    time.sleep(2)
    
    # Collect player names and filenames to keep track of what's where
    wikidatarow.append(playerfilename)
    wikidatarow.append(playerfirstname)
    wikidatarow.append(playersurname)
    wikidatarow.append(playerwikifilename)
    
    wikidata.append(wikidatarow)

# Write it out to a csv for preservation and accountability
wikidataheaders = ["Player Filename", "Firstname", "Surname", "PlayerWiki Filename"]

outfile = open('PlayerWiki/playerwikidata.csv', 'w')
csv_out = csv.writer(outfile)
csv_out.writerow(wikidataheaders)
csv_out.writerows(wikidata)
outfile.close()   

## Part 2: Finding player college teams and the Wikipedia page for the teams

In [39]:
from lxml import html

wikidata = []
with open('PlayerWiki/playerwikidata.csv', "r", newline = "") as infile:
    csvin = csv.reader(infile)
    wikidataheaders = next(csvin)
    for line in csvin:
        wikidata.append(line)

playerwikihtml = wikidataheaders.index("PlayerWiki Filename")

for playerwiki in wikidata:
    playerwikihtmlfile = playerwiki[playerwikihtml]
    infile = open(playerwikihtmlfile, "r")
    page = infile.read()
    tree = html.fromstring(page)
    
    headers = tree.xpath('//th[contains(@style,"background-color: #b0c4de")]')

    counter = 0
    for element in headers:
        if element.text == "College career":
            collegelocation = headers[counter]
            nextheader = headers[counter + 1]
        else:
            pass
        counter = counter + 1
    try:
        collegehtml = page[page.index(collegelocation.text):page.index(nextheader.text)]
        littletree=html.fromstring(collegehtml)
        collegelink = littletree.xpath('//a/@href')
        for link in collegelink:
            if link.startswith("#") == True:
                pass
            else:
                playerwiki.append(link[link.rindex('/'):])
                
    except:
        playerwiki.append("NoCollege")
        


wikidataheaders.append("First College Team Link")
wikidataheaders.append("Second College Team Link")
outfile = open('Collegeteam/collegeteamlinks.csv', 'w')
csv_out = csv.writer(outfile)
csv_out.writerow(wikidataheaders)
csv_out.writerows(wikidata)
outfile.close()    

        
    

In [7]:
# import csv

# collegelinks = []
# with open('Collegeteam/collegeteamlinks.csv', 'r', newline = "") as infile:
#     csvin = csv.reader(infile)
#     headers = next(csvin)
#     for line in csvin:       
#         collegelinks.append(line)

# finalcollegelinks = []        
# for links in collegelinks:
#     actuallinks = links[1:]
#     for actuallink in actuallinks:
#         if actuallink.startswith("#")==True:
#             pass
#         elif actuallink=="NoCollege":
#             pass
#         else:
#             finalcollegelinks.append(actuallink[actuallink.rindex('/'):]) # Only grabs the end of the link for the filename, removing the /wiki portion

# uniquecollegelinks = set(finalcollegelinks)            
# outfile = open("Collegeteam/collegeteamurls.txt", "w")
# for link in uniquecollegelinks:
#      print(link, file=outfile)
# outfile.close()



In [41]:
import requests
import time

wikidata = []
with open('Collegeteam/collegeteamlinks.csv', 'r', newline = "") as infile:
    csvin = csv.reader(infile)
    wikidataheaders = next(csvin)
    for line in csvin:       
        wikidata.append(line)
        
firstcollegeteam = wikidataheaders.index("First College Team Link")
secondcollegeteam = wikidataheaders.index("Second College Team Link")

for playerwiki in wikidata:
    try:
        secondcollege = playerwiki[secondcollegeteam]
    except:
        playerwiki.append("N/A")
    
    firstcollege = playerwiki[firstcollegeteam]
    secondcollege = playerwiki[secondcollegeteam]
    
    if firstcollege == "NoCollege":
        playerwiki.append("N/A")
    else:
        firstlinkbuild = "https://en.wikipedia.org/wiki" + firstcollege
        outfile = open('Collegeteam' + firstcollege + '.html', 'w')
        
        result = requests.get(firstlinkbuild)
        print(result.text, file=outfile)
        time.sleep(2)
        outfile.close()
    
    if secondcollege == "N/A":
        pass
    else:
        secondlinkbuild = "https://en.wikipedia.org/wiki" + secondcollege
        outfile = open('Collegeteam' + secondcollege + '.html', 'w')
        
        result = requests.get(secondlinkbuild)
        print(result.text, file=outfile)
        time.sleep(2)
        outfile.close()
    
outfile = open('Collegeteam/collegeteamlinks.csv', 'w')
csv_out = csv.writer(outfile)
csv_out.writerow(wikidataheaders)
csv_out.writerows(wikidata)
outfile.close() 

## Part 3: Finding the Wiki page for the actual university

In [58]:
from lxml import html

wikidata = []
with open('Collegeteam/collegeteamlinks.csv', 'r', newline = "") as infile:
    csvin = csv.reader(infile)
    wikidataheaders = next(csvin)
    for line in csvin:       
        wikidata.append(line)

firstcollegeteam = wikidataheaders.index("First College Team Link")
secondcollegeteam = wikidataheaders.index("Second College Team Link")

wikidataheaders.append("First College")
wikidataheaders.append("Second College")

for playerwiki in wikidata:
    
    firstcollege = playerwiki[firstcollegeteam]
    secondcollege = playerwiki[secondcollegeteam]        
        
    if firstcollege == "NoCollege":
        playerwiki.append("N/A")
    else:
        infile = open('Collegeteam' + firstcollege + ".html", 'rb')
        page = infile.read()
        tree=html.fromstring(page)    
        college = tree.xpath('//tr[contains(th,"University")]/td/a/@href')
        if college == []:
            playerwiki.append('Collegewiki' + firstcollege)
        for link in college:
            else:
                collegelink = 'Collegewiki' + link[link.rindex('/'):] # Removes the /wiki from the beginning of the url
                playerwiki.append(collegelink)
        
    if secondcollege == "N/A":
        playerwiki.append("N/A")
    else:
        infile = open('Collegeteam' + secondcollege + ".html", 'rb')
        page = infile.read()
        tree=html.fromstring(page)    
        college = tree.xpath('//tr[contains(th,"University")]/td/a/@href')
        for link in college:
            collegelink = 'Collegewiki' + link[link.rindex('/'):] # Removes the /wiki from the beginning of the url
            playerwiki.append(collegelink)
        

outfile = open('Collegewiki/collegelinks.csv', 'w')
csv_out = csv.writer(outfile)
csv_out.writerow(wikidataheaders)
csv_out.writerows(wikidata)
outfile.close()    


In [59]:
wikidata = []
with open("Collegewiki/collegelinks.csv", "r", newline = "") as infile:
    csvin = csv.reader(infile)
    wikidataheaders = next(csvin)
    for line in csvin:
        wikidata.append(line)

firstcollege = wikidataheaders.index("First College")
secondcollege = wikidataheaders.index("Second College")

for playerwiki in wikidata:
    firstcollegelink = playerwiki[firstcollege]
    secondcollegelink = playerwiki[secondcollege]
    
    if firstcollegelink == "N/A":
        pass
    else:
        firstcollegeurl = 'https://wikipedia.org/wiki' + firstcollegelink[firstcollegelink.rindex('/'):]
        
        outfile = open(firstcollegelink + '.html', 'w')
        result = requests.get(firstcollegeurl)
        print(result.text, file=outfile)
        time.sleep(2)
        outfile.close()
    
    if secondcollegelink == "N/A":
        pass
    else:
        secondcollegeurl = 'https://wikipedia.org/wiki' + secondcollegelink[secondcollegelink.rindex('/'):]
        
        outfile = open(secondcollegelink + '.html', 'w')
        result = requests.get(secondcollegeurl)
        print(result.text, file=outfile)
        time.sleep(2)
        outfile.close()        


In [72]:
wikidata = []
with open("Collegewiki/collegelinks.csv", "r", newline = "") as infile:
    csvin = csv.reader(infile)
    wikidataheaders = next(csvin)
    for line in csvin:
        wikidata.append(line)

firstcollege = wikidataheaders.index("First College")
secondcollege = wikidataheaders.index("Second College")

wikidataheaders.append("First College Name")
wikidataheaders.append("First College Public")
wikidataheaders.append("First College Private")
wikidataheaders.append("First College Community")
wikidataheaders.append("First College Location")
wikidataheaders.append("First College Enrollment")

wikidataheaders.append("Second College Name")
wikidataheaders.append("Second College Public")
wikidataheaders.append("Second College Private")
wikidataheaders.append("Second College Community")
wikidataheaders.append("Second College Location")
wikidataheaders.append("Second College Enrollment")


for playerwiki in wikidata:
    firstcollegelink = playerwiki[firstcollege]
    secondcollegelink = playerwiki[secondcollege]
    
    if firstcollegelink == "N/A":
        for loop in range(5):
            playerwiki.append("N/A")

    else:    
        infile = open(firstcollegelink + '.html', 'rb')
        page = infile.read()
        tree=html.fromstring(page)    
    
        schoolname = tree.xpath('//h1/text()')
        
        if tree.xpath('//tr[contains(th,"Type")]/td/a[contains(text(),"Public")]/text()') == []:
            publiccollege = 'n'
        else:
            publiccollege = 'y'
        
        if tree.xpath('//tr[contains(th,"Type")]/td/a[contains(text(),"Private")]/text()') == []:
            privatecollege = 'n'
        else:
            privatecollege = 'y'
            
        if tree.xpath('//tr[contains(th,"Type")]/td/a[contains(text(),"Community")]/text()') == []:
            communitycollege = 'n'
        else:
            communitycollege = 'y'
            
        students = tree.xpath('//tr[contains(th,"Students")]/td/text()')
        location = tree.xpath('//tr[contains(th,"Location")]/td/div/a/text()')    
    
        playerwiki.append(",".join(schoolname))
        playerwiki.append(publiccollege)
        playerwiki.append(privatecollege)
        playerwiki.append(communitycollege)
        playerwiki.append(",".join(location))
        playerwiki.append(",".join(students))


    if secondcollegelink == "N/A":
        for loop in range(5):
            playerwiki.append("N/A")

    else:    
        infile = open(secondcollegelink + '.html', 'rb')
        page = infile.read()
        tree=html.fromstring(page)    
    
        schoolname = tree.xpath('//h1/text()')
        
        if tree.xpath('//tr[contains(th,"Type")]/td/a[contains(text(),"Public")]/text()') == []:
            publiccollege = 'n'
        else:
            publiccollege = 'y'
        
        if tree.xpath('//tr[contains(th,"Type")]/td/a[contains(text(),"Private")]/text()') == []:
            privatecollege = 'n'
        else:
            privatecollege = 'y'
            
        if tree.xpath('//tr[contains(th,"Type")]/td/a[contains(text(),"Community")]/text()') == []:
            communitycollege = 'n'
        else:
            communitycollege = 'y'
            
        students = tree.xpath('//tr[contains(th,"Students")]/td/text()')
        location = tree.xpath('//tr[contains(th,"Location")]/td/div/a/text()')    
    
        playerwiki.append(",".join(schoolname))
        playerwiki.append(publiccollege)
        playerwiki.append(privatecollege)
        playerwiki.append(communitycollege)

        playerwiki.append(",".join(location))
        playerwiki.append(",".join(students))


outfile = open('Collegewiki/collegedata.csv', 'w')
csv_out = csv.writer(outfile)
csv_out.writerow(wikidataheaders)
csv_out.writerows(wikidata)
outfile.close()           
        
        

  